# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 8 2022 11:15AM,241398,20,ALUR273452030,Alumier Labs Inc.,Released
1,Jul 8 2022 11:15AM,241398,20,ALUR308953986,Alumier Labs Inc.,Released
2,Jul 8 2022 11:15AM,241398,20,ALUR537955632,Alumier Labs Inc.,Released
3,Jul 8 2022 11:15AM,241398,20,ALUR561197246,Alumier Labs Inc.,Released
4,Jul 8 2022 11:15AM,241398,20,ALUR770966513,Alumier Labs Inc.,Released
5,Jul 8 2022 11:15AM,241398,20,ALUR611446924,Alumier Labs Inc.,Released
6,Jul 8 2022 11:15AM,241398,20,ALUR648344167,Alumier Labs Inc.,Released
7,Jul 8 2022 11:15AM,241398,20,ALUR766089094,Alumier Labs Inc.,Released
8,Jul 8 2022 11:15AM,241398,20,ALUR822798526,Alumier Labs Inc.,Released
9,Jul 8 2022 11:15AM,241398,20,ALUR862773345,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,241391,Released,1
42,241392,Released,1
43,241395,Released,1
44,241397,Released,2
45,241398,Released,12


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241391,NaN,NaN,1.0
241392,NaN,NaN,1.0
241395,NaN,NaN,1.0
241397,NaN,NaN,2.0
241398,NaN,NaN,12.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241306,0.0,0.0,1.0
241319,32.0,9.0,2.0
241320,0.0,0.0,1.0
241326,0.0,0.0,1.0
241343,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241306,0,0,1
241319,32,9,2
241320,0,0,1
241326,0,0,1
241343,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241306,0,0,1
1,241319,32,9,2
2,241320,0,0,1
3,241326,0,0,1
4,241343,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241306,,,1
1,241319,32,9,2
2,241320,,,1
3,241326,,,1
4,241343,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 8 2022 11:15AM,241398,20,Alumier Labs Inc.
12,Jul 8 2022 11:13AM,241397,10,Emerginnova
14,Jul 8 2022 10:53AM,241395,10,"Citieffe, Inc."
15,Jul 8 2022 10:48AM,241392,19,ACG North America LLC
16,Jul 8 2022 10:35AM,241391,10,"Citieffe, Inc."
17,Jul 8 2022 10:22AM,241389,20,Sartorius Stedim North America
18,Jul 8 2022 10:22AM,241387,10,Eywa Pharma Inc.
24,Jul 8 2022 10:05AM,241386,10,Emerginnova
25,Jul 8 2022 9:52AM,241378,15,"Person & Covey, Inc."
193,Jul 8 2022 9:34AM,241381,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 8 2022 11:15AM,241398,20,Alumier Labs Inc.,,,12
1,Jul 8 2022 11:13AM,241397,10,Emerginnova,,,2
2,Jul 8 2022 10:53AM,241395,10,"Citieffe, Inc.",,,1
3,Jul 8 2022 10:48AM,241392,19,ACG North America LLC,,,1
4,Jul 8 2022 10:35AM,241391,10,"Citieffe, Inc.",,,1
5,Jul 8 2022 10:22AM,241389,20,Sartorius Stedim North America,,,1
6,Jul 8 2022 10:22AM,241387,10,Eywa Pharma Inc.,,,6
7,Jul 8 2022 10:05AM,241386,10,Emerginnova,,,1
8,Jul 8 2022 9:52AM,241378,15,"Person & Covey, Inc.",,9,159
9,Jul 8 2022 9:34AM,241381,10,ISDIN Corporation,,,39


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 11:15AM,241398,20,Alumier Labs Inc.,12,,
1,Jul 8 2022 11:13AM,241397,10,Emerginnova,2,,
2,Jul 8 2022 10:53AM,241395,10,"Citieffe, Inc.",1,,
3,Jul 8 2022 10:48AM,241392,19,ACG North America LLC,1,,
4,Jul 8 2022 10:35AM,241391,10,"Citieffe, Inc.",1,,
5,Jul 8 2022 10:22AM,241389,20,Sartorius Stedim North America,1,,
6,Jul 8 2022 10:22AM,241387,10,Eywa Pharma Inc.,6,,
7,Jul 8 2022 10:05AM,241386,10,Emerginnova,1,,
8,Jul 8 2022 9:52AM,241378,15,"Person & Covey, Inc.",159,9,
9,Jul 8 2022 9:34AM,241381,10,ISDIN Corporation,39,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 11:15AM,241398,20,Alumier Labs Inc.,12,,
1,Jul 8 2022 11:13AM,241397,10,Emerginnova,2,,
2,Jul 8 2022 10:53AM,241395,10,"Citieffe, Inc.",1,,
3,Jul 8 2022 10:48AM,241392,19,ACG North America LLC,1,,
4,Jul 8 2022 10:35AM,241391,10,"Citieffe, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 11:15AM,241398,20,Alumier Labs Inc.,12.0,NaN,NaN
1,Jul 8 2022 11:13AM,241397,10,Emerginnova,2.0,NaN,NaN
2,Jul 8 2022 10:53AM,241395,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Jul 8 2022 10:48AM,241392,19,ACG North America LLC,1.0,NaN,NaN
4,Jul 8 2022 10:35AM,241391,10,"Citieffe, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 11:15AM,241398,20,Alumier Labs Inc.,12.0,0.0,0.0
1,Jul 8 2022 11:13AM,241397,10,Emerginnova,2.0,0.0,0.0
2,Jul 8 2022 10:53AM,241395,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Jul 8 2022 10:48AM,241392,19,ACG North America LLC,1.0,0.0,0.0
4,Jul 8 2022 10:35AM,241391,10,"Citieffe, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4586102,217.0,5.0,1.0
102,723952,3.0,0.0,0.0
12,241380,10.0,0.0,0.0
15,482757,162.0,17.0,0.0
16,241373,0.0,1.0,0.0
18,241351,0.0,1.0,0.0
19,482758,11.0,23.0,3.0
20,965482,30.0,10.0,32.0
21,965383,2.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4586102,217.0,5.0,1.0
1,102,723952,3.0,0.0,0.0
2,12,241380,10.0,0.0,0.0
3,15,482757,162.0,17.0,0.0
4,16,241373,0.0,1.0,0.0
5,18,241351,0.0,1.0,0.0
6,19,482758,11.0,23.0,3.0
7,20,965482,30.0,10.0,32.0
8,21,965383,2.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,217.0,5.0,1.0
1,102,3.0,0.0,0.0
2,12,10.0,0.0,0.0
3,15,162.0,17.0,0.0
4,16,0.0,1.0,0.0
5,18,0.0,1.0,0.0
6,19,11.0,23.0,3.0
7,20,30.0,10.0,32.0
8,21,2.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,217.0
1,102,Released,3.0
2,12,Released,10.0
3,15,Released,162.0
4,16,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20,21
Status,,,,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0,3.0,32.0,0.0
Executing,5.0,0.0,0.0,17.0,1.0,1.0,23.0,10.0,2.0
Released,217.0,3.0,10.0,162.0,0.0,0.0,11.0,30.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,3.0,32.0,0.0
1,Executing,5.0,0.0,0.0,17.0,1.0,1.0,23.0,10.0,2.0
2,Released,217.0,3.0,10.0,162.0,0.0,0.0,11.0,30.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,3.0,32.0,0.0
1,Executing,5.0,0.0,0.0,17.0,1.0,1.0,23.0,10.0,2.0
2,Released,217.0,3.0,10.0,162.0,0.0,0.0,11.0,30.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()